In [1]:
import time
import cv2
import mediapipe as mp
import open3d as o3d 
import numpy as np
import copy
from constants import *
from recon_3d_hand import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
cap_l = cv2.VideoCapture("./output_L_near.mp4")
cap_r = cv2.VideoCapture("./output_R_near.mp4")

def preprocess(image: np.ndarray):
    image = cv2.undistort(image, L_INTRINSIC, L_DISTORTION)
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)
    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return results

def draw_hand(results, image):
    for hand_no, hand_landmarks in enumerate(results.multi_hand_landmarks):
        print(f'HAND NUMBER: {hand_no+1}')
        print('-----------------------')
        print(hand_landmarks)
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())

hands = mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)
    
# skip first 2 seconds
for i in range(60):
    cap_l.read()
    cap_r.read()

success_l, image_l = cap_l.read()
success_r, image_r = cap_r.read()
img_h = image_l.shape[0]
img_w = image_l.shape[1]

results_l = preprocess(image_l)
results_r = preprocess(image_r)

cv2.imwrite("image_l.jpg", image_l)
cv2.imwrite("image_r.jpg", image_r)

log = open("log.txt", "w")
print("image_l", file=log)
hand_landmarks_l, hand_landmarks_r = None, None
for hand_no, hand_landmarks in enumerate(results_l.multi_hand_landmarks):
    print(f'HAND NUMBER: {hand_no+1}')
    print('-----------------------')
    print(hand_landmarks, file=log)
print()

print("image_r", file=log)
for hand_no, hand_landmarks in enumerate(results_r.multi_hand_landmarks):
    print(f'HAND NUMBER: {hand_no+1}')
    print('-----------------------')
    print(hand_landmarks, file=log)

log.close()

cap_l.release()
cap_r.release()

HAND NUMBER: 1
-----------------------

HAND NUMBER: 1
-----------------------


In [3]:
results_l.multi_hand_landmarks

[landmark {
   x: 0.5119716
   y: 0.46511874
   z: 5.013267e-09
 }
 landmark {
   x: 0.49663305
   y: 0.42661428
   z: -0.014274559
 }
 landmark {
   x: 0.4700387
   y: 0.400224
   z: -0.022088015
 }
 landmark {
   x: 0.44542393
   y: 0.3867668
   z: -0.02916176
 }
 landmark {
   x: 0.42720497
   y: 0.37633583
   z: -0.037117977
 }
 landmark {
   x: 0.43202165
   y: 0.43580496
   z: -0.013240789
 }
 landmark {
   x: 0.3992369
   y: 0.42970952
   z: -0.020881668
 }
 landmark {
   x: 0.37862957
   y: 0.42743522
   z: -0.02657172
 }
 landmark {
   x: 0.36152437
   y: 0.4265106
   z: -0.03106045
 }
 landmark {
   x: 0.43126878
   y: 0.46360937
   z: -0.01344802
 }
 landmark {
   x: 0.39608142
   y: 0.46863395
   z: -0.019876318
 }
 landmark {
   x: 0.3750423
   y: 0.47221562
   z: -0.026504908
 }
 landmark {
   x: 0.3583005
   y: 0.47534367
   z: -0.032198947
 }
 landmark {
   x: 0.4382964
   y: 0.48684642
   z: -0.015277623
 }
 landmark {
   x: 0.4087016
   y: 0.49703792
   z: -0.02421399

In [4]:
hand_2d_l = map_hand_to_2d(results_l, img_w, img_h)
hand_2d_r = map_hand_to_2d(results_r, img_w, img_h)

In [5]:
hand_2d_r

[array([[128.79201889, 262.47467995],
        [118.45064163, 243.92097473],
        [100.61891556, 231.22787476],
        [ 84.24786568, 224.84381676],
        [ 72.11100578, 219.4754076 ],
        [ 75.76274395, 248.33639145],
        [ 55.74825287, 245.50658226],
        [ 43.44602108, 244.71745491],
        [ 33.28650475, 244.74615097],
        [ 74.84756947, 261.91884041],
        [ 53.16241264, 264.06211853],
        [ 40.51186085, 266.05361938],
        [ 30.62821388, 268.20227623],
        [ 78.45066547, 273.64445686],
        [ 59.38266754, 277.72773743],
        [ 48.05189133, 280.61525345],
        [ 39.04788256, 283.03645134],
        [ 85.33477783, 283.93672943],
        [ 71.14654064, 289.78048325],
        [ 62.6199007 , 292.67014503],
        [ 55.13030529, 294.93424416]])]

In [6]:
hand_3d = recon_3d_hand_points(image_l, image_r, results_l, results_r)

In [8]:
hand_3d

array([[ 4.33493112e+00, -9.86865261e-01, -9.85363868e+00,
        -1.77599841e+01, -2.34360671e+01, -2.18304561e+01,
        -3.18463335e+01, -3.79371350e+01, -4.28656499e+01,
        -2.19638776e+01, -3.26379711e+01, -3.88000001e+01,
        -4.35896372e+01, -1.96648709e+01, -2.86835058e+01,
        -3.46127957e+01, -3.95023604e+01, -1.54576684e+01,
        -2.09763670e+01, -2.42789103e+01, -2.71034857e+01],
       [-2.65284023e-02, -1.11578746e+01, -1.86083221e+01,
        -2.22324327e+01, -2.51305508e+01, -8.19397394e+00,
        -9.74394177e+00, -1.01915360e+01, -1.02177548e+01,
        -2.13565104e-01,  1.19657376e+00,  2.32291776e+00,
         3.42296751e+00,  6.55120222e+00,  9.17964490e+00,
         1.08955252e+01,  1.22223592e+01,  1.23957560e+01,
         1.60668494e+01,  1.77585833e+01,  1.89459978e+01],
       [ 3.59192183e+02,  3.55975910e+02,  3.50241938e+02,
         3.44809552e+02,  3.40652757e+02,  3.42073192e+02,
         3.35658479e+02,  3.31661165e+02,  3.28294490e

In [9]:
pointcloud, lines = connect_3d_hand_points(hand_3d)

In [12]:
o3d.visualization.draw_geometries([pointcloud, lines])